In [2]:
import pandas as pd
import openpyxl
import warnings
import os

warnings.filterwarnings("ignore")

In [10]:
base_path = "BASE.xlsx"
df_base = pd.read_excel(base_path)
df_base = df_base.iloc[:, 3:] # Eliminar columnas iniciales
df_base = df_base.iloc[7:, :] # Eliminar filas iniciales
df_base = df_base.rename(columns=df_base.iloc[0]) # Renombra las columnas
df_base = df_base[1:]
df_base = df_base.reset_index(drop=True) # Reinicia los indices
df_base = df_base.dropna(subset=["Cuenta","ACC"]) # Elimina filas con valores NaN
df_base = df_base.reset_index(drop=True)
columnas_deseadas = ["ACC", "Cuenta", "Fecha doc.", "Demora", "     Importe en ML"]
df_base = df_base[columnas_deseadas]
df_base = df_base.rename(columns={"Fecha doc.":"Fecha","     Importe en ML": "Importe"})
df_base["Fecha"] = pd.to_datetime(df_base["Fecha"], format="%d.%m.%Y")
df_base["Demora"] = df_base["Demora"].astype("Int64")
df_base["Importe"] = df_base["Importe"].astype(float)
# Condition 1
df_base["Status"] = df_base["Importe"].apply(lambda x: "DEUDA" if x > 0 else "SALDOS A FAVOR")
# Condition 2
df_base["Tipo Deuda"] = df_base["Demora"].apply(lambda x: "CORRIENTE" if x <= 0 else "VENCIDA")
# Condition 3
df_base["Saldo Final"] = df_base.apply(lambda row: row["Importe"] if (row["Status"] == "DEUDA" and row["Tipo Deuda"] == "VENCIDA") else (row["Importe"] if row["Status"] == "SALDOS A FAVOR" else "NO"), axis=1)
df_base = df_base[df_base["Saldo Final"] != "NO"]
df_base = df_base.sort_values(by=["Cuenta"], ascending=[True])
df_base = df_base.sort_values(by=["ACC"], ascending=[True])
df_base = df_base.sort_values(by=["Demora"], ascending=[False])
df_base = df_base.reset_index(drop=True)

ultima_fila = df_base.shape[0]
print(df_base["Saldo Final"].sum())

for i in range(ultima_fila):
    if df_base.loc[i, "Status"] == "DEUDA":
        saldoDeuda = df_base.loc[i, "Saldo Final"]
        for j in range(ultima_fila):
            if (
                df_base.loc[i, "Cuenta"]    == df_base.loc[j, "Cuenta"] and 
                df_base.loc[i, "ACC"]       == df_base.loc[j, "ACC"]    and 
                df_base.loc[j, "Status"]    == "SALDOS A FAVOR"
                ):
                saldoFavor = df_base.loc[j, "Saldo Final"]
                montoCompensar = min(saldoDeuda, abs(saldoFavor))
                df_base.loc[i, "Saldo Final"] = saldoDeuda - montoCompensar
                df_base.loc[j, "Saldo Final"] = saldoFavor + montoCompensar
                saldoDeuda = df_base.loc[i, "Saldo Final"]



print(df_base["Saldo Final"].sum())
print(df_base.iloc[371,0])
print(df_base.dtypes)
print(df_base.columns)
print(df_base.shape)
df_base.head(15)

150730.27999999982
     ACC   Cuenta      Fecha  Demora   Importe          Status Tipo Deuda  \
0   PE10  4073002 2018-05-07    2233     -7.24  SALDOS A FAVOR    VENCIDA   
1   PE01  4073002 2019-08-27    1660     -0.59  SALDOS A FAVOR    VENCIDA   
2   PE16  4073005 2020-08-28    1300   1589.79           DEUDA    VENCIDA   
3   PE16  4073005 2020-08-28    1300     64.99           DEUDA    VENCIDA   
4   PE17  4073005 2020-10-30    1237     34.67           DEUDA    VENCIDA   
5   PE16  4073005 2021-04-06    1207   3478.00           DEUDA    VENCIDA   
6   PE16  4073005 2020-12-29    1177     16.25           DEUDA    VENCIDA   
7   PE16  4073005 2021-01-28    1147    588.80           DEUDA    VENCIDA   
8   PE16  4073005 2021-01-29    1146     46.14           DEUDA    VENCIDA   
9   PE16  4073005 2021-01-29    1146  28751.30           DEUDA    VENCIDA   
10  PE16  4070150 2021-12-15    1146  76277.27           DEUDA    VENCIDA   
11  PE16  4073005 2021-02-25    1119   3007.47           

,ACC,Cuenta,Fecha,Demora,Importe,Status,Tipo Deuda,Saldo Final
0,PE10,4073002,2018-05-07,2233,-7.24,SALDOS A FAVOR,VENCIDA,-7.24
1,PE01,4073002,2019-08-27,1660,-0.59,SALDOS A FAVOR,VENCIDA,-0.59
2,PE16,4073005,2020-08-28,1300,1589.79,DEUDA,VENCIDA,1589.79
3,PE16,4073005,2020-08-28,1300,64.99,DEUDA,VENCIDA,64.99
4,PE17,4073005,2020-10-30,1237,34.67,DEUDA,VENCIDA,34.67
5,PE16,4073005,2021-04-06,1207,3478.00,DEUDA,VENCIDA,3478.0
6,PE16,4073005,2020-12-29,1177,16.25,DEUDA,VENCIDA,16.25
7,PE16,4073005,2021-01-28,1147,588.80,DEUDA,VENCIDA,588.8
8,PE16,4073005,2021-01-29,1146,46.14,DEUDA,VENCIDA,46.14
9,PE16,4073005,2021-01-29,1146,28751.30,DEUDA,VENCIDA,28751.3
